In [ ]:
import requests
import pandas as pd
import numpy as np
from IPython.display import display

In [ ]:
# used to get lap data on a desired car and no of laps
def get_lapData(session, car_no, start, end):
    print(f'Lap data for car No. {car_no} is on its way!!')
    data_list = []

    if session == '':
        session = 'latest'
    
    for i in range(start, end):
        response = requests.get(f'https://api.openf1.org/v1/laps?session_key={session}&driver_number={car_no}&lap_number={i}')

        if response.status_code != 200:
            return 'API request error now. Try again!'
        
        data = response.json()
        
        if len(data) > 0:
            data_list.append(data[0])

    return data_list

# function used to clean lap data
def clean_lapData(df):
    df.drop(columns=['meeting_key', 'session_key', 'date_start'])
    df.rename(columns={"duration_sector_1": "S1", "duration_sector_2": "S2", "duration_sector_3" : 'S3'}, inplace=True)
    return df


In [ ]:
# used to convert the list data into a df
def to_df(data):
    return pd.DataFrame(data)

# used to add data to csv
def into_csv(df, file_path):
    return df.to_csv(f'{file_path}', index=False)
    

In [ ]:
# used to get info on stints
def get_stint(session, car_no, tyre_age):
    print(f'Stint data for car No. {car_no} is incoming!!')

    data_list = []

    if session == '':
        session = 'latest'
    
    response = requests.get(f'https://api.openf1.org/v1/stints?session_key={session}&tyre_age_at_start>={tyre_age}')

    if response.status_code != 200:
        return 'API request error currently. Review info and try again!!'
    
    data = response.json()

    for i in range(0, len(data)):
        if data[i]['driver_number'] == car_no: 
            data_list.append(data[i])
            
    return data_list

# function used to clean stint data
def clean_stintData(df): 
    df.drop(columns=['meeting_key', 'session_key'])
    return df

In [ ]:
# used to get data relating to car performance 
def get_carData(session, car_no):
    print(f'Car data for car No. {car_no} is on its way!!')

    data_list = []

    if session == '':
        session = 'latest'

    response = requests.get(f'https://api.openf1.org/v1/car_data?driver_number={car_no}&session_key={session}&speed>=330')

    if response.status_code != 200:
        return 'API request error currently. Review info and try again!!'
    
    data = response.json()

    for i in range(0, len(data)):
        if data[i]['driver_number'] == car_no: 
            data_list.append(data[i])
            
    return data_list

In [ ]:
# used to get session key and correlate fruther to retrive lap data
def get_session(country, year, type):
    # type options: Race, Sprint, Qualifying, Pratice (1, 2, 3)
    print(f'Getting session info for {country} {year}, {type}!!')

    response = requests.get(f'https://api.openf1.org/v1/sessions?country_name={country}&session_name={type}&year={year}')

    if response.status_code != 200:
        return 'API request error currently. Review info and try again!!'

    data = response.json()

    data_list = {
        'session': data[0]['session_key'],
        'meeting': data[0]['meeting_key']
    }

    return data_list


In [ ]:
# Full code example in use, i.e. 1. (REAL)

# 1. State the session you want to get information for!
session  = get_session('Belgium', '2024', 'Race')

# 2. Get stint data for the race, with session key just found...
stint_data = get_stint(session['session'], 44, 0)

# 3. Unpack the desired stint data to retrive the laps
start = stint_data[0]['lap_start']
end = stint_data[0]['lap_end'] + 1

# 4. feed the stint lap data, into the full get lap data method
lap_data = get_lapData(session['session'], 44, start + 1, end)  # +1 is needed to avoid api error

# 5. conver the lap data into a df for futher anylsis
lap_df = to_df(lap_data)

# turn it into a csv
csv = into_csv(lap_df, 'Data/Test.csv')


Getting session info for Belgium 2024, Race!!
Stint data for car No. 44 is incoming!!
Lap data for car No. 44 is on its way!!
This CSV is going into location:  Data/Test.csv
None


In [195]:
# Full code example in use, i.e. 3. (COPY)

# 1. State the session you want to get information for!
session  = get_session('Belgium', '2024', 'Race')

# 2. get all the drivers numbers, and clean the data
car_no = pd.read_html('https://www.motorsport.com/f1/news/f1-race-numbers-2023-which-driver-uses-what-number/10404332/', index_col=False)[0]
car_no = np.array(car_no.iloc[:, :1]).flatten()

# 3. display the stints for each of the drivers
for car in car_no:
    stint_info = get_stint(session['session'], car, 0)
    stint_df = to_df(stint_info)
    stint_df = stint_df[stint_df['compound'] == 'HARD'].reset_index(drop=True)

    # stint_no = np.array(stint_df.loc[:, 'stint_number':'stint_number']).flatten()
    # stint_df.reset_index(drop=True, inplace=True)
    # print(stint_df)

    display(stint_df)
    print('len is: ', len(stint_data))

    data_list = []
    for i in range(0, len(stint_df)):
        start = stint_df['lap_start']
        end = stint_df['lap_end']
        print('start: ', start, ' end: ', end)
        lap = get_lapData(session['session'], car, start, end)
        data_list.append(lap)
    
    lap_data = to_df(data_list)

    display('hi')

Getting session info for Belgium 2024, Race!!
Stint data for car No. 1 is incoming!!


,meeting_key,session_key,stint_number,driver_number,lap_start,lap_end,compound,tyre_age_at_start
0,1242,9574,2,1,11,28,HARD,0


len is:  3
start:  0    11
Name: lap_start, dtype: int64  end:  0    28
Name: lap_end, dtype: int64
Lap data for car No. 1 is on its way!!


TypeError: 'Series' object cannot be interpreted as an integer

In [196]:
# Full code example in use, i.e. 1. (REAL)

# 1. State the session you want to get information for!
session  = get_session('Belgium', '2024', 'Race')

# 2. Get stint data for the race, with session key just found...
stint_data = get_stint(session['session'], 44, 0)

# 3. Unpack the desired stint data to retrive the laps
start = stint_data[0]['lap_start']
end = stint_data[0]['lap_end'] + 1

# 4. feed the stint lap data, into the full get lap data method
lap_data = get_lapData(session['session'], 44, start + 1, end)  # +1 is needed to avoid api error

# 5. conver the lap data into a df for futher anylsis
lap_df = to_df(lap_data)

# 5.5 apply the needed methods to clean the lap df
lap_df = clean_lapData(lap_df)

lap_df

Getting session info for Belgium 2024, Race!!
Stint data for car No. 44 is incoming!!
Lap data for car No. 44 is on its way!!


,meeting_key,session_key,driver_number,i1_speed,i2_speed,st_speed,date_start,lap_duration,is_pit_out_lap,S1,S2,S3,segments_sector_1,segments_sector_2,segments_sector_3,lap_number
0,1242,9574,44,329.0,202,307.0,2024-07-28T13:05:45.881000+00:00,109.849,False,31.284,49.211,29.354,"[2049, 2049, 2048, 2049, 2048, 2048, 2049, 2049]","[2049, 2049, 2051, 2049, 2048, 2049, 2049, 204...","[2048, 2048, 2048, 2048, 2048, 2051, 0]",2
1,1242,9574,44,326.0,204,310.0,2024-07-28T13:07:35.621000+00:00,109.109,False,31.215,48.486,29.408,"[None, 2048, 2049, 2049, 2049, 2049, 2049, 2048]","[2049, 2049, 2048, 2048, 2049, 2049, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2051, 2048]",3
2,1242,9574,44,305.0,203,305.0,2024-07-28T13:09:24.787000+00:00,109.808,False,31.754,48.605,29.449,"[2048, 2051, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2051, 2051, 2048, 2048, 2051, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",4
3,1242,9574,44,307.0,201,307.0,2024-07-28T13:11:14.724000+00:00,109.376,False,31.592,48.320,29.464,"[2049, 2051, 2048, 2048, 2048, 2048, 2048, 2048]","[2049, 2048, 2051, 2048, 2048, 2051, 2048, 204...","[2049, 2048, 2048, 2048, 2048, 2049, 2048]",5
4,1242,9574,44,308.0,204,NaN,2024-07-28T13:13:04.006000+00:00,109.400,False,31.618,48.355,29.427,"[None, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2051, 2048, 2048, 2051, 2049, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",6
5,1242,9574,44,309.0,202,306.0,2024-07-28T13:14:53.395000+00:00,109.357,False,31.662,48.124,29.571,"[2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2049, 2051, 2048, 2049, 2048, 2048, 2049, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",7
6,1242,9574,44,NaN,200,NaN,2024-07-28T13:16:42.767000+00:00,109.666,False,31.646,48.515,29.505,"[None, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2049, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",8
7,1242,9574,44,NaN,200,307.0,2024-07-28T13:18:32.516000+00:00,109.704,False,31.616,48.572,29.516,"[None, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",9
8,1242,9574,44,NaN,200,NaN,2024-07-28T13:20:22.216000+00:00,109.951,False,31.625,48.752,29.574,"[None, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2049, 2064]",10
9,1242,9574,44,311.0,202,306.0,2024-07-28T13:22:12.078000+00:00,114.328,False,31.618,48.726,33.984,"[None, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2049, 2049, 2049, 2049, 2048, 2051]",11


In [ ]:
# Full code example in use, i.e. 1. (REAL)

# 1. State the session you want to get information for!
session  = get_session('Belgium', '2024', 'Race')

# 2. Get stint data for the race, with session key just found...
stint_data = get_stint(session['session'], 44, 0)

# 3. Unpack the desired stint data to retrive the laps
start = stint_data[0]['lap_start']
end = stint_data[0]['lap_end'] + 1

# 4. feed the stint lap data, into the full get lap data method
lap_data = get_lapData(session['session'], 44, start + 1, end)  # +1 is needed to avoid api error

# 5. conver the lap data into a df for futher anylsis
lap_df = to_df(lap_data)

# 5.5 apply the needed methods to clean the lap df
lap_df = clean_lapData(lap_df)

lap_df

Getting session info for Belgium 2024, Race!!
Stint data for car No. 44 is incoming!!
Lap data for car No. 44 is on its way!!


,meeting_key,session_key,driver_number,i1_speed,i2_speed,st_speed,date_start,lap_duration,is_pit_out_lap,S1,S2,S3,segments_sector_1,segments_sector_2,segments_sector_3,lap_number
0,1242,9574,44,329.0,202,307.0,2024-07-28T13:05:45.881000+00:00,109.849,False,31.284,49.211,29.354,"[2049, 2049, 2048, 2049, 2048, 2048, 2049, 2049]","[2049, 2049, 2051, 2049, 2048, 2049, 2049, 204...","[2048, 2048, 2048, 2048, 2048, 2051, 0]",2
1,1242,9574,44,326.0,204,310.0,2024-07-28T13:07:35.621000+00:00,109.109,False,31.215,48.486,29.408,"[None, 2048, 2049, 2049, 2049, 2049, 2049, 2048]","[2049, 2049, 2048, 2048, 2049, 2049, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2051, 2048]",3
2,1242,9574,44,305.0,203,305.0,2024-07-28T13:09:24.787000+00:00,109.808,False,31.754,48.605,29.449,"[2048, 2051, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2051, 2051, 2048, 2048, 2051, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",4
3,1242,9574,44,307.0,201,307.0,2024-07-28T13:11:14.724000+00:00,109.376,False,31.592,48.320,29.464,"[2049, 2051, 2048, 2048, 2048, 2048, 2048, 2048]","[2049, 2048, 2051, 2048, 2048, 2051, 2048, 204...","[2049, 2048, 2048, 2048, 2048, 2049, 2048]",5
4,1242,9574,44,308.0,204,NaN,2024-07-28T13:13:04.006000+00:00,109.400,False,31.618,48.355,29.427,"[None, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2051, 2048, 2048, 2051, 2049, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",6
5,1242,9574,44,309.0,202,306.0,2024-07-28T13:14:53.395000+00:00,109.357,False,31.662,48.124,29.571,"[2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2049, 2051, 2048, 2049, 2048, 2048, 2049, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",7
6,1242,9574,44,NaN,200,NaN,2024-07-28T13:16:42.767000+00:00,109.666,False,31.646,48.515,29.505,"[None, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2049, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",8
7,1242,9574,44,NaN,200,307.0,2024-07-28T13:18:32.516000+00:00,109.704,False,31.616,48.572,29.516,"[None, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",9
8,1242,9574,44,NaN,200,NaN,2024-07-28T13:20:22.216000+00:00,109.951,False,31.625,48.752,29.574,"[None, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2049, 2064]",10
9,1242,9574,44,311.0,202,306.0,2024-07-28T13:22:12.078000+00:00,114.328,False,31.618,48.726,33.984,"[None, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2049, 2049, 2049, 2049, 2048, 2051]",11


In [ ]:
# Full code example in use, i.e. 2. (REAL)
data = []
car_no = [44, 63]

# 1. State the session you want to get information for!
session  = get_session('Belgium', '2024', 'Race')

for car in car_no:
    # 2. Get stint data for the race, with session key just found...
    stint_data = get_stint(session['session'], car, 0)

    # 3. Unpack the desired stint data to retrive the laps
    start = stint_data[len(stint_data) - 1]['lap_start']
    end = stint_data[len(stint_data) - 1]['lap_end'] + 1

    # 4. feed the stint lap data, into the full get lap data method
    lap_data = get_lapData(session['session'], car, start, end)  # +1 is needed to avoid api error, for initial stint

    # 5. conver the lap data into a df for futher anylsis
    lap_df = to_df(lap_data)

    # 5.5 apply the needed methods to clean the lap df
    lap_df = clean_lapData(lap_df)

    # 6. add it to a csv
    into_csv(lap_df)

    # 7. display the df's
    display(lap_df)

Getting session info for Belgium 2024, Race!!
Stint data for car No. 44 is incoming!!
Lap data for car No. 44 is on its way!!


,meeting_key,session_key,driver_number,i1_speed,i2_speed,st_speed,date_start,lap_duration,is_pit_out_lap,S1,S2,S3,segments_sector_1,segments_sector_2,segments_sector_3,lap_number
0,1242,9574,44,325.0,208.0,310.0,2024-07-28T13:51:28.893000+00:00,121.300,True,45.393,46.984,28.923,"[2064, 2064, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2049, 2049, 2049, 204...","[2049, 2049, 2048, 2049, 2048, 2048, 2048]",27
1,1242,9574,44,332.0,206.0,313.0,2024-07-28T13:53:30.084000+00:00,107.396,False,30.835,47.762,28.799,"[2048, 2049, 2049, 2049, 2049, 2049, 2049, 2049]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2049, 2048]",28
2,1242,9574,44,311.0,207.0,310.0,2024-07-28T13:55:17.526000+00:00,107.429,False,31.059,47.387,28.983,"[2049, 2048, 2049, 2048, 2048, 2048, 2048, 2048]","[2049, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2049, 2048, 2048, 2048, 2048, 2049, 2048]",29
3,1242,9574,44,314.0,209.0,NaN,2024-07-28T13:57:04.977000+00:00,107.031,False,31.176,46.961,28.894,"[2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2049, 2048, 2049, 2049, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",30
4,1242,9574,44,315.0,206.0,NaN,2024-07-28T13:58:51.905000+00:00,107.182,False,31.181,47.047,28.954,"[None, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2049, 2048, 204...","[2049, 2048, 2048, 2048, 2048, 2048, 2048]",31
5,1242,9574,44,NaN,211.0,311.0,2024-07-28T14:00:39.129000+00:00,107.003,False,31.156,46.903,28.944,"[None, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2049, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",32
6,1242,9574,44,314.0,207.0,NaN,2024-07-28T14:02:26.166000+00:00,106.653,False,31.053,46.607,28.993,"[None, 2048, 2049, 2048, 2048, 2048, 2048, 2048]","[2049, 2048, 2049, 2048, 2048, 2049, 2049, 204...","[2048, 2048, 2048, 2048, 2048, 2049, 2048]",33
7,1242,9574,44,318.0,206.0,312.0,2024-07-28T14:04:12.921000+00:00,106.856,False,30.993,46.933,28.930,"[None, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2049, 2049, 2048, 2048, 2048, 2048, 2049, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",34
8,1242,9574,44,317.0,208.0,313.0,2024-07-28T14:05:59.782000+00:00,106.729,False,31.062,46.667,29.000,"[None, 2048, 2049, 2048, 2049, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",35
9,1242,9574,44,318.0,206.0,312.0,2024-07-28T14:07:46.546000+00:00,106.931,False,30.984,46.986,28.961,"[None, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",36


Stint data for car No. 63 is incoming!!
Lap data for car No. 63 is on its way!!


,meeting_key,session_key,driver_number,i1_speed,i2_speed,st_speed,date_start,lap_duration,is_pit_out_lap,S1,S2,S3,segments_sector_1,segments_sector_2,segments_sector_3,lap_number
0,1242,9574,63,344.0,194.0,315.0,2024-07-28T13:22:23.707000+00:00,122.456,True,44.917,48.672,28.867,"[2064, 2064, 2048, 2048, 2048, 2049, 2049, 2049]","[2049, 2048, 2048, 2048, 2049, 2048, 2049, 204...","[2049, 2048, 2048, 2049, 2048, 2048, 2048]",11
1,1242,9574,63,337.0,201.0,314.0,2024-07-28T13:24:26.229000+00:00,109.872,False,30.730,49.781,29.361,"[2048, 2049, 2049, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2049, 2048, 2048, 2048, 2048, 2048, 2048]",12
2,1242,9574,63,330.0,202.0,315.0,2024-07-28T13:26:16.084000+00:00,108.702,False,30.791,48.868,29.043,"[None, 2048, 2049, 2049, 2049, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2049, 2048, 2048, 2049, 2048, 2048, 2048]",13
3,1242,9574,63,333.0,203.0,NaN,2024-07-28T13:28:04.744000+00:00,108.265,False,30.840,48.403,29.022,"[None, 2048, 2049, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2049, 2048, 204...","[2049, 2048, 2048, 2049, 2048, 2048, 2048]",14
4,1242,9574,63,326.0,204.0,NaN,2024-07-28T13:29:52.972000+00:00,108.315,False,30.900,48.400,29.015,"[None, 2048, 2048, 2049, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2049, 2051, 204...","[2049, 2048, 2048, 2048, 2048, 2048, 2048]",15
5,1242,9574,63,328.0,204.0,317.0,2024-07-28T13:31:41.297000+00:00,108.606,False,30.885,48.680,29.041,"[None, 2048, 2048, 2048, 2049, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",16
6,1242,9574,63,335.0,204.0,313.0,2024-07-28T13:33:29.900000+00:00,108.460,False,30.833,48.575,29.052,"[None, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2049, 2048, 2048, 2048, 2048, 204...","[2049, 2048, 2048, 2048, 2048, 2048, 2048]",17
7,1242,9574,63,NaN,205.0,NaN,2024-07-28T13:35:18.344000+00:00,108.556,False,30.944,48.483,29.129,"[None, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2049, 204...","[2048, 2048, 2048, 2048, 2049, 2048, 2048]",18
8,1242,9574,63,333.0,203.0,315.0,2024-07-28T13:37:06.965000+00:00,108.499,False,30.951,48.379,29.169,"[None, 2048, 2048, 2048, 2049, 2048, 2048, 2048]","[2048, 2049, 2049, 2048, 2048, 2048, 2051, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",19
9,1242,9574,63,338.0,203.0,314.0,2024-07-28T13:38:55.429000+00:00,109.605,False,31.005,49.248,29.352,"[2051, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",20
